## Neural Net - 30 Second features

In [20]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd

In [21]:
## Dropping / cleaning data


df_30_sec = pd.read_csv("../data/features_30_sec.csv")


print("Shape all data:", df_30_sec.shape)
# print(df_30_sec.head())

# Encode Labels
le = LabelEncoder()
df_30_sec['label_encoded'] = le.fit_transform(df_30_sec['label'])


print("\nCategories Encoded:")
for i, category in enumerate(le.classes_):
    print(f"{category}: {i}")
    
## Splitting / Scaling data 
y = df_30_sec["label_encoded"].to_numpy()
X = df_30_sec.drop(columns=["filename", "length", "label", "label_encoded"]).to_numpy()

[X_train, X_test, y_train, y_test] = train_test_split(X, y, test_size = .2, random_state=42)

print("\nTrain Size:", len(X_train[:]))
print("Test Size:", len(X_test[:]))

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(X_test)




Shape all data: (1000, 60)

Categories Encoded:
blues: 0
classical: 1
country: 2
disco: 3
hiphop: 4
jazz: 5
metal: 6
pop: 7
reggae: 8
rock: 9

Train Size: 800
Test Size: 200
[[ 0.68221918  0.06206651 -1.25032485 ...  0.06234857  0.94596436
  -0.26275888]
 [ 0.91808318 -0.17398442  1.65686311 ... -0.24380684 -0.78406413
  -0.38632968]
 [-1.05044203 -0.13185441  1.02940764 ...  0.01051284 -0.24352654
   0.83540992]
 ...
 [-0.27273291 -0.48032228  2.16192977 ...  0.18069975  1.94893433
   0.07370814]
 [ 3.19286233 -5.63096899 -0.67257058 ... -1.30557956  1.28591132
  -1.17127214]
 [ 0.45360574 -0.48357081  1.91492653 ... -0.53359154 -0.78125525
  -0.78827039]]


## Building  Model

In [22]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)



in_size = X_train.shape[1]
print("Input size:", in_size)

class Net(nn.Module):
    def __init__(self, in_size, num_classes=10):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_size, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            
            nn.Linear(64, 32),
            nn.ReLU(),

            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        return self.layers(x)

model_NN = Net(in_size)

Input size: 57


## Training Model

In [23]:


criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_NN.parameters(), lr=0.05)

num_epochs = 5000

for i in range(num_epochs):
    y_pred = model_NN(X_train_tensor)
    loss = criterion(y_pred, y_train_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    
    if (i+1) % 200 == 0:
        print(f'Epoch [{i+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [200/5000], Loss: 1.5192
Epoch [400/5000], Loss: 0.8963
Epoch [600/5000], Loss: 0.5922
Epoch [800/5000], Loss: 0.4252
Epoch [1000/5000], Loss: 0.3137
Epoch [1200/5000], Loss: 0.2307
Epoch [1400/5000], Loss: 0.1579
Epoch [1600/5000], Loss: 0.1145
Epoch [1800/5000], Loss: 0.0906
Epoch [2000/5000], Loss: 0.0830
Epoch [2200/5000], Loss: 0.0703
Epoch [2400/5000], Loss: 0.0593
Epoch [2600/5000], Loss: 0.0464
Epoch [2800/5000], Loss: 0.0377
Epoch [3000/5000], Loss: 0.0388
Epoch [3200/5000], Loss: 0.0300
Epoch [3400/5000], Loss: 0.0325
Epoch [3600/5000], Loss: 0.0342
Epoch [3800/5000], Loss: 0.0270
Epoch [4000/5000], Loss: 0.0230
Epoch [4200/5000], Loss: 0.0242
Epoch [4400/5000], Loss: 0.0239
Epoch [4600/5000], Loss: 0.0146
Epoch [4800/5000], Loss: 0.0202
Epoch [5000/5000], Loss: 0.0176


## Evaluating model

In [9]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

with torch.no_grad():
    y_pred = model_NN(X_test_tensor)
    _, predicted = torch.max(y_pred, dim=1)
    accuracy = (predicted == y_test_tensor).float().mean()
    print(f'Test Accuracy: {accuracy.item():.4f}')

Test Accuracy: 0.7100


In [19]:
print("\nClassification report:")
print(classification_report(predicted, y_test, digits=4))


Classification report:
              precision    recall  f1-score   support

           0     0.8500    0.6800    0.7556        25
           1     0.9231    1.0000    0.9600        12
           2     0.6296    0.6800    0.6538        25
           3     0.6190    0.5909    0.6047        22
           4     0.7333    0.6111    0.6667        18
           5     0.8636    0.8636    0.8636        22
           6     0.8800    0.8800    0.8800        25
           7     0.6923    0.6923    0.6923        13
           8     0.4783    0.5789    0.5238        19
           9     0.5238    0.5789    0.5500        19

    accuracy                         0.7100       200
   macro avg     0.7193    0.7156    0.7150       200
weighted avg     0.7196    0.7100    0.7123       200

